# OnlineStats.jl

https://github.com/joshday/OnlineStats.jl

Online algorithms for statistics.

Josh Day (`@joshday`)

In [179]:
using OnlineStats, Plots
gr()

Plots.GRBackend()

# Motivation 

Suppose:
- Your data doesn't fit in RAM
- Your data is streaming
    - real-time
    - You get a batch once/day

![](http://www.ibmbigdatahub.com/sites/default/files/styles/xlarge-scaled/public/infographic_image/4-Vs-of-big-data.jpg?itok=4syrvSLX)

- Statisticians don't have tools to handle all of this
- Adapting our favorite algorithms is often nontrivial

# Types of problems
- Easy 
    - updates the parameter directly (mean, variance)
- Hard
    - updates "sufficient statistics" from which parameter is calculated (linear regression)
- Impossible
    - No analytical solution, must be approximated (quantile regression)

# Introducing OnlineStats.jl

- Accepts input data piece by piece, not all at once
- Algorithms use O(1) memory
- Provides functionality for major areas of statistics
    - summary statistics to penalized regression

# Why Julia

- The same familiar story

# Toy Example

In [180]:
using OnlineStats
y = randn(100)

o = Mean()      # create

for yi in y
    fit!(o, yi) # update
end

# It works
mean(y) - mean(o)

5.898059818321144e-17

# Each OnlineStat is a type

In [181]:
# Construct objects with data
x = randn(100, 2)
Means(x)

■ Means{EqualWeight}
  >     value: [0.08906268171889252,-0.026647955133385758]
  >      nobs: 100


In [182]:
# Or create "empty" object, add data later
Means(2)

■ Means{EqualWeight}
  >     value: [0.0,0.0]
  >      nobs: 0


# Types are parameterized by Weight

- Last argument of constructor
- Default depends on whether problem is Easy/Hard vs. Impossible

In [183]:
Variance(EqualWeight())

■ Variance{EqualWeight}
  >     value: 0.0
  >      nobs: 0


- Consider online mean (many updates have this form):

$$\theta^{(t)} = (1 - \gamma_t) \; \theta^{(t-1)} + \gamma_t \; x_t$$

- For $\gamma_t = \frac{1}{t}$, updates are equivalent to offline mean
    - `EqualWeight`

- ExponentialWeight
    - $\gamma_t = \lambda$
- BoundedEqualWeight 
    - start as EqualWeight, shift to ExponentialWeight
    - $\gamma_t = \text{max}\left(\frac{1}{t}, \lambda\right)$
- LearningRate
    - Main use is for stochastic approximation
    - $\gamma_t = \text{max}\left(\frac{1}{t^r}, λ\right)$

In [199]:
o1 = EqualWeight()
o2 = ExponentialWeight(.2)
o3 = BoundedEqualWeight(.2)
o4 = LearningRate(.5)
ovec = [o1, o2, o3, o4]
map(OnlineStats.updatecounter!, ovec)

plt_wt = plot([0], map(OnlineStats.weight, ovec)', w = 3,
label = ["EqualWeight" "ExponentialWeight(.2)" "BoundedEqualWeight(.2)" "LearningRate(.5)"],
xlabel = "nobs", ylabel = "weight value", layout = 4, ylims = (0, 1))
for i in 1:50
    for o in ovec
        OnlineStats.updatecounter!(o)
    end
    push!(plt_wt, nobs(o1), map(OnlineStats.weight, ovec))
end

In [200]:
plt_wt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 
 
 
 nobs 
 
 
 weight value 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 62.5219,15.748 71.0628,86.4556 75.3332,110.025 79.6037,121.809 83.8741,128.88 88.1446,133.594 92.415,136.961 96.6855,139.486 100.956,141.45 105.226,143.022 
 109.497,144.307 113.767,145.379 118.038,146.285 122.308,147.062 126.579,147.735 130.849,148.325 135.12,148.845 139.39,149.307 143.66,149.72 147.931,150.092 
 152.201,150.429 156.472,150.735 160.742,151.015 165.013,151.271 169.283,151.507 173.554,151.724 177.824,151.926 182.095,152.113 186.365,152.287 190.635,152.449 
 194.906,152.601 199.176,152.744 203.447,152.878 207.717,153.004 211.988,153.123 216.258,153.235 220.529,153.341 224.799,153.442 229.07,153.537 233.34,153.628 
 237.61,153.714 241.881,153.796 246.151,153.874 250.422,153.949 254.692,154.021 258.963,154.089 263.233,154.154 267.504,154.217 271.774,154.277 276.045,154.335 
 280.315,154.39 
 "/>
 
 
 
 
 EqualWeight 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 
 
 
 nobs 
 
 
 weight value 
 
<polyline clip-path="url(#clip03)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 362.522,128.88 371.063,128.88 375.333,128.88 379.604,128.88 383.874,128.88 388.145,128.88 392.415,128.88 396.685,128.88 400.956,128.88 405.226,128.88 
 409.497,128.88 413.767,128.88 418.038,128.88 422.308,128.88 426.579,128.88 430.849,128.88 435.12,128.88 439.39,128.88 443.66,128.88 447.931,128.88 
 452.201,128.88 456.472,128.88 460.742,128.88 465.013,128.88 469.283,128.88 473.554,128.88 477.824,128.88 482.095,128.88 486.365,128.88 490.635,128.88 
 494.906,128.88 499.176,128.88 503.447,128.88 507.717,128.88 511.988,128.88 516.258,128.88 520.529,128.88 524.799,128.88 529.07,128.88 533.34,128.88 
 537.61,128.88 541.881,128.88 546.151,128.88 550.422,128.88 554.692,128.88 558.963,128.88 563.233,128.88 567.504,128.88 571.774,128.88 576.045,128.88 
 580.315,128.88 
 "/>
 
 
 
 
 ExponentialWeight(.2) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 
 
 
 nobs 
 
 
 weight value 
 
<polyline clip-path="url(#clip04)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 62.5219,215.748 71.0628,286.456 75.3332,310.025 79.6037,321.809 83.8741,328.88 88.1446,328.88 92.415,328.88 96.6855,328.88 100.956,328.88 105.226,328.88 
 109.497,328.88 113.767,328.88 118.038,328.88 122.308,328.88 126.579,328.88 130.849,328.88 135.12,328.88 139.39,328.88 143.66,328.88 147.931,328.88 
 152.201,328.88 156.472,328.88 160.742,328.88 165.013,328.88 169.283,328.88 173.554,328.88 177.824,328.88 182.095,328.88 186.365,328.88 190.635,328.88 
 194.906,328.88 199.176,328.88 203.447,328.88 207.717,328.88 211.988,328.88 216.258,328.88 220.529,328.88 224.799,328.88 229.07,328.88 233.34,328.88 
 237.61,328.88 241.881,328.88 246.151,328.88 250.422,328.88 254.692,328.88 258.963,328.88 263.233,328.88 267.504,328.88 271.774,328.88 276.045,328.88 
 280.315,328.88 
 "/>
 
 
 
 
 BoundedEqualWeight(.2) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 
 
 
 nobs 
 
 
 weight value 
 
<polyline clip-path="url(#clip05)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 362.522,215.748 371.063,257.168 375.333,275.517 379.604,286.456 383.874,293.92 388.145,299.431 392.415,303.713 396.685,307.165 400.956,310.025 405.226,312.444 
 409.497,314.525 413.767,316.34 418.038,317.942 422.308,319.368 426.579,320.65 430.849,321.809 435.12,322.865 439.39,323.831 443.66,324.72 447.931,325.542 
 452.201,326.304 456.472,327.013 460.742,327.676 465.013,328.297 469.283,328.88 473.554,329.429 477.824

In [186]:
y = -cumsum(rand(1000)) + 30 * randn(1000)
o1 = Mean(EqualWeight())
o2 = Mean(ExponentialWeight(.1))
plt_mean1 = plot([0], [0.0 0.0], label = ["EqualWeight" "ExponentialWeight"], w = 2, 
xlab = "time", ylab = "value", title = "changing model", legend = false)
map_rows(10, y) do yi
    fit!(o1, yi)
    fit!(o2, yi)
    push!(plt_mean1, nobs(o1), [mean(o1), mean(o2)])
end
scatter!(plt_mean1, y, markersize = 1);

y = 30 * randn(1000)
o1 = Mean(EqualWeight())
o2 = Mean(ExponentialWeight(.1))
plt_mean2 = plot([0], [0.0 0.0], label = ["EqualWeight" "ExponentialWeight"], w = 2, 
xlab = "time", ylab = "value", title = "constant model", legend = false)
map_rows(10, y) do yi
    fit!(o1, yi)
    fit!(o2, yi)
    push!(plt_mean2, nobs(o1), [mean(o1), mean(o2)])
end
scatter!(plt_mean2, y, markersize = 1);


# The Weight matters

In [198]:
plot(plt_mean1, plt_mean2, size = (800, 400))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 - 
 
 
 400 
 
 
 - 
 
 
 200 
 
 
 0 
 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 
 
 
 changing model 
 
 
 time 
 
 
 value 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 71.516,80.2796 74.5141,85.3766 77.5121,83.3441 80.5102,82.5463 83.5082,85.3328 86.5063,86.9686 89.5043,86.2176 92.5024,86.6905 95.5004,89.1828 98.4984,90.1948 
 101.496,91.2724 104.495,92.7821 107.493,94.3944 110.491,95.3098 113.489,96.2546 116.487,97.6084 119.485,98.7841 122.483,99.5219 125.481,101.024 128.479,102.539 
 131.477,103.452 134.475,104.754 137.473,105.687 140.471,106.487 143.469,107.617 146.467,108.445 149.465,109.577 152.463,110.671 155.461,111.604 158.459,112.589 
 161.457,113.686 164.456,114.841 167.454,116.004 170.452,117.038 173.45,118.22 176.448,119.499 179.446,120.619 182.444,121.724 185.442,122.771 188.44,123.648 
 191.438,124.72 194.436,125.983 197.434,127.399 200.432,128.377 203.43,129.634 206.428,130.788 209.426,131.856 212.424,133.104 215.422,134.323 218.42,135.61 
 221.418,136.739 224.416,137.939 227.415,139.059 230.413,140.301 233.411,141.464 236.409,142.573 239.407,143.864 242.405,145.021 245.403,146.246 248.401,147.348 
 251.399,148.554 254.397,149.708 257.395,150.884 260.393,152.013 263.391,153.244 266.389,154.459 269.387,155.436 272.385,156.619 275.383,157.832 278.381,158.931 
 281.379,160.016 284.377,161.077 287.375,162.19 290.374,163.295 293.372,164.388 296.37,165.62 299.368,166.76 302.366,167.958 305.364,169.111 308.362,170.366 
 311.36,171.579 314.358,172.723 317.356,173.796 320.354,174.92 323.352,176.101 326.35,177.208 329.348,178.395 332.346,179.56 335.344,180.697 338.342,181.816 
 341.34,182.909 344.338,183.97 347.336,185.165 350.334,186.357 353.333,187.495 356.331,188.618 359.329,189.742 362.327,190.961 365.325,192.23 368.323,193.275 
 371.321,194.443 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 71.516,80.2796 74.5141,83.7922 77.5121,82.3601 80.5102,81.0503 83.5082,89.9228 86.5063,90.6291 89.5043,85.3984 92.5024,87.4806 95.5004,99.8646 98.4984,98.5765 
 101.496,99.7525 104.495,105.085 107.493,109.767 110.491,106.946 113.489,109.852 116.487,114.765 119.485,114.775 122.483,112.907 125.481,121.117 128.479,126.103 
 131.477,123.34 134.475,129.574 137.473,125.018 140.471,124.355 143.469,128.661 146.467,128.553 149.465,135.357 152.463,139.434 155.461,137.405 158.459,141.244 
 161.457,143.468 164.456,148.511 167.454,150.672 170.452,150.402 173.45,154.84 176.448,159.637 179.446,158.007 182.444,160.286 185.442,161.597 188.44,158.748 
 191.438,163.442 194.436,170.178 197.434,179.585 200.432,173.478 203.43,179.084 206.428,181.542 209.426,181.772 212.424,188.376 215.422,191.055 218.42,195.726 
 221.418,193.289 224.416,198.024 227.415,196.215 230.413,200.666 233.411,203.607 236.409,203.088 239.407,210.399 242.405,209.646 245.403,215.329 248.401,213.889 
 251.399,215.753 254.397,218.179 257.395,221.986 260.393,222.157 263.391,227.93 266.389,231.907 269.387,223.357 272.385,231.182 275.383,237.178 278.381,234.105 
 281.379,234.631 284.377,235.837 287.375,240.356 290.374,241.125 293.372,243.997 296.37,253.155 299.368,252.593 302.366,255.584 305.364,258.028 308.362,264.616 
 311.36,265.56 314.358,264.306 317.356,262.637 320.354,266.242 323.352,269.868 326.35,270.105 329.348,275.327 332.346,278.793 335.344,278.474 338.342,280.55 
 341.34,280.704 344.338,279.618 347.336,288.43 350.334,293.354 353.333,293.318 356.331,293.607 359.329,296.617 362.327,304.923 365.325,311.576 368.323,301.688 
 371.321,307.706 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


# Methods

- The following methods are available for every OnlineStat:
    - `fit!(o, data...)`
    - `nobs(o)`
    - `value(o)`
- Most types also have more methods:

In [188]:
o = LinReg(5, intercept = false)
coef(o)'

1x5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0

# Updating

In [189]:
y = randn(100)
o = Mean()

for yi in y
    fit!(o, y)
end

fit!(o, y)      # simpler version of for loop

fit!(o, y, 10)  # update in batches of size 10

fit!(o, y, 0.1) # use update weight of .1 for each update

w = rand(100)
fit!(o, y, w)   # observation y[i] gets weight w[i]
;

# Why so many updating methods?

- Best seen through example
    - Linear regression using stochastic gradient descent
    - Consider plot of coefficients over time

In [190]:
using DataGenerator
x, y, β = linregdata(10_000, 5);

In [191]:
# "standard" update
o = StatLearn(size(x, 2), SGD(), intercept = false)
plt_sgd1 = plot([0], coef(o)', lab = ["b$i" for i in 1:5]', xlab = "nobs", ylab = "value")
map_rows(100, x, y) do xi, yi
    fit!(o, xi, yi); push!(plt_sgd1, nobs(o), coef(o))
end

# batch update
o = StatLearn(size(x, 2), SGD(), intercept = false)
plt_sgd2 = plot([0], coef(o)', lab = ["b$i" for i in 1:5]', xlab = "nobs", ylab = "value")
map_rows(100, x, y) do xi, yi
    fit!(o, xi, yi, 50); push!(plt_sgd2, nobs(o), coef(o))
end

# override weight
o = StatLearn(size(x, 2), SGD(), intercept = false)
plt_sgd3 = plot([0], coef(o)', lab = ["b$i" for i in 1:5]', xlab = "nobs", ylab = "value")
map_rows(100, x, y) do xi, yi
    fit!(o, xi, yi, .001); push!(plt_sgd3, nobs(o), coef(o))
end

In [192]:
# fit!(o, x, y)
plt_sgd1

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 - 
 
 
 1 
 
 
 0 
 
 
 1 
 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 
 
 
 nobs 
 
 
 value 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.5219,175.323 67.6998,357.163 72.8777,279.673 78.0557,301.953 83.2336,281.406 88.4115,315.43 93.5895,298.727 98.7674,310.753 103.945,299.79 109.123,293.745 
 114.301,324.243 119.479,341.588 124.657,295.449 129.835,302.393 135.013,269.513 140.191,305.408 145.369,313.591 150.547,318.804 155.725,286.135 160.903,290.067 
 166.08,319.38 171.258,324.162 176.436,322.775 181.614,316.584 186.792,325.747 191.97,325.912 197.148,308.825 202.326,304.234 207.504,308.024 212.682,314.072 
 217.86,310.876 223.038,322.984 228.216,341.458 233.394,314.964 238.572,326.979 243.749,303.183 248.927,322.899 254.105,311.723 259.283,302.598 264.461,301.88 
 269.639,301.902 274.817,306.999 279.995,287.843 285.173,291.538 290.351,308.456 295.529,293.514 300.707,309.146 305.885,307.815 311.063,305.394 316.24,299.66 
 321.418,296.705 326.596,295.371 331.774,285.488 336.952,299.133 342.13,305.48 347.308,312.687 352.486,303.249 357.664,288.265 362.842,313.429 368.02,303.274 
 373.198,298.991 378.376,296.445 383.554,313.556 388.732,299.648 393.909,301.911 399.087,311.562 404.265,287.46 409.443,299.593 414.621,282.116 419.799,301.754 
 424.977,299.006 430.155,307.98 435.333,298.208 440.511,289.053 445.689,298.216 450.867,306.396 456.045,308.307 461.223,303.608 466.4,312.017 471.578,310.356 
 476.756,304.907 481.934,295.57 487.112,300.187 492.29,314.485 497.468,322.797 502.646,303.364 507.824,301.421 513.002,331.512 518.18,320.217 523.358,313.974 
 528.536,311.568 533.714,313.452 538.892,317.108 544.069,319.272 549.247,318.596 554.425,309.882 559.603,307.889 564.781,303.69 569.959,288.453 575.137,304.108 
 580.315,311.895 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.5219,175.323 67.6998,264.473 72.8777,250.705 78.0557,263.865 83.2336,221.021 88.4115,256.43 93.5895,239.282 98.7674,259.495 103.945,236.189 109.123,233.251 
 114.301,280.435 119.479,249.132 124.657,264.881 129.835,268.127 135.013,234.708 140.191,255.933 145.369,263.095 150.547,253.373 155.725,236.737 160.903,253.528 
 166.08,239.256 171.258,249.311 176.436,231.07 181.614,231.126 186.792,233.296 191.97,269.22 197.148,200.334 202.326,234.807 207.504,230.07 212.682,249.118 
 217.86,243.181 223.038,238.385 228.216,241.067 233.394,241.662 238.572,232.771 243.749,262.678 248.927,253.236 254.105,249.748 259.283,242.772 264.461,217.954 
 269.639,232.177 274.817,239.658 279.995,228.908 285.173,238.016 290.351,226.277 295.529,253.221 300.707,235.914 305.885,247.336 311.063,244.195 316.24,238.2 
 321.418,234.887 326.596,247.716 331.774,253.719 336.952,241.399 342.13,243.715 347.308,234.931 352.486,230.813 357.664,242.641 362.842,245.178 368.02,230.716 
 373.198,238.389 378.376,254.335 383.554,252.609 388.732,246.03 393.909,243.208 399.087,226.776 404.265,235.212 409.443,236.722 414.621,238.621 419.799,249.277 
 424.977,254.207 430.155,258.973 435.333,264.711 440.511,253.882 445.689,245.369 450.867,226.521 456.045,234.512 461.223,244.121 466.4,248.118 471.578,239.795 
 476.756,235.085 481.934,254.424 487.112,265.226 492.29,229.376 497.468,229.481 502.646,229.422 507.824,243.361 513.002,260.095 518.18,253.875 523.358,260.291 
 528.536,242.094 533.714,248.679 538.892,252.109 544.069,233.917 549.247,234.776 554.425,229.263 559.603,221.566 564.781,228.304 569.959,252.609 575.137,247.759 
 580.315,243.538 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#3cb370; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.5219,175.323 67.6998,140.127 72.8777,163.393 78.0557,196.092 83.2336,119.752 88.4115,152.422 93.5895,188.446 98.7674,181.919 103.945,211.742 109.123,178.863 
 114.301,182.422 119.479,179.

In [193]:
# fit!(o, x, y, 50)
plt_sgd2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 - 
 
 
 1.0 
 
 
 - 
 
 
 0.5 
 
 
 0 
 
 
 0.5 
 
 
 1.0 
 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 
 
 
 nobs 
 
 
 value 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.5219,176.178 67.6998,357.163 72.8777,310.151 78.0557,317.483 83.2336,322.379 88.4115,335.217 93.5895,321.894 98.7674,337.694 103.945,327.496 109.123,320.871 
 114.301,328.099 119.479,335.302 124.657,332.948 129.835,329.21 135.013,322.49 140.191,324.302 145.369,330.986 150.547,333 155.725,323.572 160.903,319.743 
 166.08,326.89 171.258,330.986 176.436,332.723 181.614,333.885 186.792,336.683 191.97,335.966 197.148,328.652 202.326,326.274 207.504,326.878 212.682,327.431 
 217.86,331.765 223.038,333.948 228.216,338.045 233.394,336.139 238.572,340.17 243.749,335.431 248.927,334.984 254.105,332.012 259.283,329.279 264.461,329.643 
 269.639,327.682 274.817,327.323 279.995,321.982 285.173,319.572 290.351,321.582 295.529,321.313 300.707,322.838 305.885,326.042 311.063,324.508 316.24,324.586 
 321.418,323.065 326.596,321.546 331.774,315.24 336.952,316.826 342.13,317.019 347.308,319.849 352.486,319.802 357.664,317.155 362.842,319.833 368.02,322.424 
 373.198,321.016 378.376,320.721 383.554,325.097 388.732,323.687 393.909,322.538 399.087,324.337 404.265,320.142 409.443,321.176 414.621,316.644 419.799,320.09 
 424.977,319.928 430.155,322.6 435.333,319.404 440.511,316.493 445.689,315.207 450.867,316.506 456.045,318.103 461.223,318.353 466.4,319.794 471.578,320.845 
 476.756,321.608 481.934,319.26 487.112,320.062 492.29,322.638 497.468,325.476 502.646,323.981 507.824,323.102 513.002,329.399 518.18,330.549 523.358,330.607 
 528.536,329.324 533.714,329.912 538.892,330.988 544.069,331.711 549.247,331.39 554.425,329.548 559.603,327.906 564.781,325.143 569.959,321.671 575.137,322.243 
 580.315,322.792 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.5219,176.178 67.6998,249.083 72.8777,252.419 78.0557,268.864 83.2336,267.848 88.4115,257.692 93.5895,253.446 98.7674,251.104 103.945,255.261 109.123,246.769 
 114.301,252.677 119.479,249.591 124.657,259.34 129.835,265.944 135.013,255.416 140.191,258.974 145.369,259.771 150.547,261.095 155.725,257.415 160.903,255.49 
 166.08,251.434 171.258,252.452 176.436,255.765 181.614,254.442 186.792,253.135 191.97,263.48 197.148,246.673 202.326,248.157 207.504,245.447 212.682,246.581 
 217.86,246.04 223.038,245.726 228.216,246.098 233.394,247.634 238.572,247.395 243.749,254.447 248.927,254.504 254.105,255.539 259.283,254.308 264.461,246.703 
 269.639,247.575 274.817,247.325 279.995,246.27 285.173,248.709 290.351,244.596 295.529,249.239 300.707,247.682 305.885,251.742 311.063,252.539 316.24,250.773 
 321.418,250.75 326.596,253.09 331.774,254.47 336.952,253.981 342.13,253.321 347.308,251.789 352.486,248.392 357.664,250.046 362.842,250.811 368.02,248.713 
 373.198,249.366 378.376,253.228 383.554,254.219 388.732,253.827 393.909,253.273 399.087,250.331 404.265,249.955 409.443,248.352 414.621,248.003 419.799,249.982 
 424.977,252.788 430.155,254.327 435.333,257.078 440.511,255.398 445.689,254.851 450.867,250.669 456.045,250.269 461.223,251.182 466.4,251.849 471.578,250 
 476.756,248.224 481.934,254.365 487.112,257.123 492.29,250.801 497.468,248.819 502.646,248.187 507.824,249.471 513.002,254.729 518.18,255.259 523.358,257.582 
 528.536,255.177 533.714,255.585 538.892,255.657 544.069,251.67 549.247,250.993 554.425,248.731 559.603,244.803 564.781,242.836 569.959,247.275 575.137,248.016 
 580.315,247.803 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#3cb370; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.5219,176.178 67.6998,155.083 72.8777,177.625 78.0557,170.556 83.2336,167.347 88.4115,162.394 93.5895,182.236 98.7674,177.706 103.945,183.889 109.123,181.954 
 11

In [194]:
# fit!(o, x, y, .001)
plt_sgd3

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 - 
 
 
 1.0 
 
 
 - 
 
 
 0.5 
 
 
 0 
 
 
 0.5 
 
 
 1.0 
 
 
 
 0 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 
 
 
 nobs 
 
 
 value 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.5219,187.003 67.6998,209.743 72.8777,222.504 78.0557,235.288 83.2336,246.801 88.4115,259.658 93.5895,268.329 98.7674,281.289 103.945,285.461 109.123,290.49 
 114.301,297.307 119.479,303.83 124.657,308.883 129.835,313.568 135.013,315.705 140.191,320.019 145.369,324.706 150.547,328.495 155.725,327.948 160.903,329.071 
 166.08,333.165 171.258,336.611 176.436,338.764 181.614,340.731 186.792,343.121 191.97,345.251 197.148,344.237 202.326,344.458 207.504,345.55 212.682,346.5 
 217.86,349.214 223.038,350.798 228.216,353.914 233.394,354.401 238.572,357.163 243.749,355.812 248.927,356.078 254.105,355.177 259.283,354.321 264.461,354.588 
 269.639,353.711 274.817,353.586 279.995,350.917 285.173,349.408 290.351,349.922 295.529,349.673 300.707,350.191 305.885,351.781 311.063,350.983 316.24,351.004 
 321.418,350.101 326.596,349.209 331.774,345.448 336.952,345.984 342.13,345.783 347.308,347.104 352.486,346.895 357.664,345.151 362.842,346.499 368.02,348.006 
 373.198,347.116 378.376,346.85 383.554,349.45 388.732,348.672 393.909,348.115 399.087,349.273 404.265,346.763 409.443,347.257 414.621,344.296 419.799,346.269 
 424.977,346.126 430.155,347.9 435.333,345.778 440.511,343.776 445.689,342.687 450.867,343.338 456.045,344.213 461.223,344.32 466.4,345.238 471.578,345.943 
 476.756,346.567 481.934,345.001 487.112,345.504 492.29,347.278 497.468,349.292 502.646,348.468 507.824,347.951 513.002,352.479 518.18,353.713 523.358,354.028 
 528.536,353.34 533.714,354.044 538.892,355.046 544.069,355.843 549.247,355.78 554.425,354.638 559.603,353.614 564.781,351.553 569.959,348.867 575.137,349.18 
 580.315,349.473 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.5219,187.003 67.6998,191.001 72.8777,201.786 78.0557,210.046 83.2336,215.842 88.4115,222.505 93.5895,226.948 98.7674,231.112 103.945,235.013 109.123,235.909 
 114.301,238.759 119.479,241.573 124.657,246.452 129.835,250.874 135.013,250.986 140.191,254.966 145.369,257.332 150.547,260.161 155.725,260.85 160.903,261.661 
 166.08,261.334 171.258,261.905 176.436,264.789 181.614,265.281 186.792,265.313 191.97,270.128 197.148,264.794 202.326,265.354 207.504,264.49 212.682,264.779 
 217.86,264.391 223.038,264.25 228.216,264.273 233.394,265.211 238.572,265.184 243.749,268.206 248.927,268.667 254.105,269.654 259.283,269.606 264.461,266.199 
 269.639,266.556 274.817,266.256 279.995,265.63 285.173,266.807 290.351,264.704 295.529,266.859 300.707,266.052 305.885,268.27 311.063,268.993 316.24,268.291 
 321.418,268.419 326.596,269.739 331.774,270.654 336.952,270.556 342.13,270.353 347.308,269.669 352.486,267.749 357.664,268.643 362.842,269.152 368.02,267.937 
 373.198,267.979 378.376,270.288 383.554,271.004 388.732,270.998 393.909,270.678 399.087,268.936 404.265,268.684 409.443,267.582 414.621,267.29 419.799,268.51 
 424.977,270.272 430.155,271.339 435.333,273.336 440.511,272.374 445.689,272.15 450.867,269.424 456.045,269.104 461.223,269.652 466.4,270.123 471.578,268.926 
 476.756,267.633 481.934,271.759 487.112,273.77 492.29,269.619 497.468,268.205 502.646,267.581 507.824,268.369 513.002,272.048 518.18,272.662 523.358,274.493 
 528.536,272.971 533.714,273.368 538.892,273.556 544.069,270.803 549.247,270.292 554.425,268.499 559.603,265.41 564.781,263.606 569.959,266.68 575.137,267.138 
 580.315,266.872 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#3cb370; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.5219,187.003 67.6998,185.319 72.8777,188.889 78.0557,188.306 83.2336,185.577 88.4115,185.607 93.5895,188.909 98.7674,186.187 103.945,188.857 109.123,187.986 


# Stochastic approximation

# StatLearn

Models of the form

$$\frac{1}{n} \sum_{i=1}^n \ell_i(\theta) + g(\theta)$$

Objective is sum of 
- `Model` (linear regression) and
- `Penalty` (LASSO)

```julia
StatLearn{
    M <: Model, 
    P <: Penalty, 
    A <: Algorithm, 
    W <: Weight
}
```
- Algorithms are variants of stochastic gradient descent
- Trace plot examples used `L2Regression()` with `NoPenalty()` and `SGD()`

**StatLearn can fit any combination of `Model`, `Penalty`, and `Algorithm`**

- L2Regression, L1Regression, PoissonRegression, LogisticRegression, SVMLike, HuberRegression, QuantileRegression
- RidgePenalty, LassoPenalty, ElasticNetPenalty
- SGD, AdaGrad, AdaDelta, FOBOS, ADAM, MMGrad (my experimental algorithm)

In [195]:
o = StatLearn(5, LogisticRegression(), AdaGrad(), RidgePenalty(.1))

■ StatLearn
  >     value: [0.0,0.0,0.0,0.0,0.0,0.0]
  >     model: LogisticRegression
  >   penalty: RidgePenalty (λ = 0.1)
  > algorithm: AdaGrad
  >      nobs: 0


# Experimental features
- Bootstrap

# Future Work

- OnlineStats assumes observations are in rows
    - Standard notation, but inefficient with column-major storage
- Not clear from names which types are approximate, which can be exact
- Easier trace plots